In [16]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import json
import pandas
from pandas import DataFrame
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_md')
from string import punctuation
import itertools

In [17]:
from collections import defaultdict

In [28]:
def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))
def get_ingredientScript(url):
    jsonld = get_ld_json(url)
    useful = jsonld[1]
    ingredients = useful["recipeIngredient"]
    return ingredients
def parse_ingredients(url):
    ingredients = get_ingredientScript(url)
    df = DataFrame (ingredients,columns=['ingredients'])
    # 2 cases
    # "number" ("()") ("unit") (adjective) "noun/subject - ingredient" (, other)
    # contains "to taste"
    df = df["ingredients"]
    df_taste = df[df.str.contains('to taste', case = False)]
    
    df_unit = df[~df.str.contains('to taste', case = False)]
    
    # array of arrays: each array is amount, unit, ingredient, descriptor, preparation
    ingredients_parsed = []
    
    for i in df:
        curr_arr = ["", "", "", "", ""]
        print(i)
        if 'to taste' not in i:
        
            # before we look at POS, remove everything after the comma and put it in prep
            split_string = i.split(", ", 1)
            root_phrase = split_string[0]
            if len(split_string) > 1:
                other_piece = split_string[1]
            else:
                other_piece = ""

            curr_arr[4] = other_piece

            split_string2 = root_phrase.split("(", 1)
            if len(split_string2) > 1:
                split_string3 = split_string2[1].split(")", 1)
                curr_arr[3] = split_string3[0]
                root_phrase = split_string2[0].strip() + split_string3[1]
            doc = nlp(root_phrase)
            index = 0
            for token in doc:
                found_num = False
                # only get first number if it matches criteria
                #if found_num == False and token.pos_ == "NOUN" and not token.is_alpha or token.pos_ == "NUM":
                if index == 0:
                    curr_arr[0] = token.text
                elif index == 1:
                    if token.pos_ != "ADJ":
                        curr_arr[1] = token.text
                    else:
                        curr_arr[3] = token.text
                elif token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else: 
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
                index+=1
        else:
            i = i.replace("to taste", "")
            doc = nlp(i)
            curr_arr[0] = "to taste"
            for token in doc:
                if token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else:
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
        ingredients_parsed.append(curr_arr)
        #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
        #displacy.render(doc, style="dep") # change to serve when we go to python


    
    return ingredients_parsed

In [46]:
url1 = "https://www.allrecipes.com/recipe/283483"

In [47]:
json_ld = get_ld_json(url1)

In [48]:
ing = parse_ingredients(url1)
print(ing)

½ cup unsalted butter, at room temperature
¾ cup white sugar
2 teaspoons vanilla sugar
1 large lemon, zested and juiced
4 large eggs, divided
2 cups blanched almond flour
9 ounces ricotta cheese
⅓ cup white sugar
¼ cup sliced almonds, or to taste
1 tablespoon confectioners' sugar for dusting
[['½', 'cup', 'butter', 'unsalted', 'at room temperature'], ['¾', 'cup', '', 'white sugar', ''], ['2', 'teaspoons', 'sugar', 'vanilla', ''], ['1', '', 'lemon', 'large', 'zested and juiced'], ['4', '', 'eggs', 'large', 'divided'], ['2', 'cups', 'blanched', 'almond flour', ''], ['9', 'ounces', 'cheese', 'ricotta', ''], ['⅓', 'cup', '', 'white sugar', ''], ['to taste', '', 'sliced', '¼ cup almonds , or', ''], ['1', 'tablespoon', 'sugar', "confectioners ' for dusting", '']]


In [49]:
one_list = itertools.chain.from_iterable(ing)
fil = list(dict.fromkeys(one_list))
new = ['mixture','ingredients']
name = json_ld[1]['name']
for ing in fil:
    for word in ing.split():
        new.append(word)
for w in name.lower().split():
    new.append(w)
new = list(dict.fromkeys(new))
print(new)

['mixture', 'ingredients', '½', 'cup', 'butter', 'unsalted', 'at', 'room', 'temperature', '¾', 'white', 'sugar', '2', 'teaspoons', 'vanilla', '1', 'lemon', 'large', 'zested', 'and', 'juiced', '4', 'eggs', 'divided', 'cups', 'blanched', 'almond', 'flour', '9', 'ounces', 'cheese', 'ricotta', '⅓', 'to', 'taste', 'sliced', '¼', 'almonds', ',', 'or', 'tablespoon', 'confectioners', "'", 'for', 'dusting', 'cake']


In [50]:
text = json_ld[1]['recipeInstructions']
if len(text) > 1:
    new_text = ''
    for i,t in enumerate(text):
        new_text = new_text + text[i]['text']
    text = new_text
else:
    text = text[0]['text']
print(text)
print(name)

Preheat the oven to 325 degrees F (165 degrees C). Grease an 8-inch springform pan and line with parchment paper.
Combine butter and 3/4 cup sugar in a large bowl; beat with an electric mixer until creamy, about 3 minutes. Add vanilla sugar, lemon zest, and lemon juice; mix until well combined. Add egg yolks, 1 at a time, beating well after each addition.
Fold in almond flour and ricotta cheese with a silicone spatula.
Wash and dry the beaters and beat egg whites in a glass, metal, or ceramic bowl until foamy. Gradually add 1/3 cup sugar, continuing to beat until stiff peaks form. Fold stiff egg whites into the batter. Pour batter into the prepared pan. Sprinkle with sliced almonds.
Bake on the center rake of the preheated oven until edges of the cake are lightly browned, about 50 minutes. Cake will still be a little wobbly in the middle.
Remove from the oven and let the cake to cool completely in the pan before transferring to a cake platter, about 1 hour. Dust with confectioners' sug

In [51]:
doc = nlp('Spread 2 cups of the sauce mixture in the bottom of the baking dish'.lower())
for token in doc:
    print(token.text,token.pos_, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children], token.lemma_)
displacy.render(doc, style="dep")

spread NOUN ROOT spread NOUN [cups, in] spread
2 NUM nummod cups NOUN [] 2
cups NOUN dobj spread NOUN [2, of] cup
of ADP prep cups NOUN [mixture] of
the DET det mixture NOUN [] the
sauce NOUN compound mixture NOUN [] sauce
mixture NOUN pobj of ADP [the, sauce] mixture
in ADP prep spread NOUN [bottom] in
the DET det bottom NOUN [] the
bottom NOUN pobj in ADP [the, of] bottom
of ADP prep bottom NOUN [dish] of
the DET det dish NOUN [] the
baking NOUN compound dish NOUN [] baking
dish NOUN pobj of ADP [the, baking] dish


In [52]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text,chunk.root.head.pos_)

spread spread ROOT spread NOUN
2 cups cups dobj spread NOUN
the sauce mixture mixture pobj of ADP
the bottom bottom pobj in ADP
the baking dish dish pobj of ADP


In [53]:
def find_tool(sentence):
    pattern = [{"LEMMA":{"IN":['in','with']}},{"POS":'DET'},{"POS":'NOUN', 'OP':'+'}]
    # tool_pattern = [{"POS":'NOUN', 'OP':'+'}]
    tools = []
    matcher = Matcher(nlp.vocab)
    matcher.add('find', [pattern])
    doc = nlp(sentence)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        for token in span:
            if token.pos_ == 'NOUN':
                tools.append(token)
    return tools
find_tool('Heat butter in a pan over medium heat')
def find_tool2(sentence):
    in_list = ['in','into','on','to']
    too =[]
    not_tools = []
    sent = nlp(sentence.lower())
    for chunk in sent.noun_chunks:
#         and chunk.root.head.pos_ != 'VERB' and not chunk.root.is_sent_start
        if chunk.root.text not in new and chunk.root.head.text in in_list:
            too.append(chunk.text)
#     for token in sent:
#         if token.text not in too and token.pos_ == 'NOUN':
#             not_tools.append(token.text)
    return too, not_tools
        
    

In [54]:
tools = []

for sentence in nlp(text).sents:
    print(sentence)
    t,nt = find_tool2(str(sentence))
    for each in t:
        print(each)
#         if each not in big_not_tools:
        tools.append(each)
print(tools)
    
    
    

Preheat the oven to 325 degrees F (165 degrees C).
325 degrees
Grease an 8-inch springform pan and line with parchment paper.

Combine butter and 3/4 cup sugar in a large bowl; beat with an electric mixer until creamy, about 3 minutes.
a large bowl
Add vanilla sugar, lemon zest, and lemon juice; mix until well combined.
Add egg yolks, 1 at a time, beating well after each addition.

Fold in almond flour and ricotta cheese with a silicone spatula.

Wash and dry the beaters and beat egg whites in a glass, metal, or ceramic bowl until foamy.
a glass, metal, or ceramic bowl
Gradually add 1/3 cup sugar, continuing to beat until stiff peaks form.
Fold stiff egg whites into the batter.
the batter
Pour batter into the prepared pan.
the prepared pan
Sprinkle with sliced almonds.

Bake on the center rake of the preheated oven until edges of the cake are lightly browned, about 50 minutes.
the center rake
Cake will still be a little wobbly in the middle.

the middle
Remove from the oven and let the